In [7]:
from collections import defaultdict
import pandas as pd
from Bio import SearchIO
import glob


In [4]:
# set list of attributes needed
attribs = ['accession', 'bias', 'bitscore', 'description', 'cluster_num', 'domain_exp_num',  'domain_included_num', 'domain_obs_num', 'domain_reported_num', 'env_num', 'evalue', 'id', 'overlap_num', 'region_num']


In [18]:
# define all hmm outputs
hmm_out = glob.glob("./hmm_out/*.tab.out")

In [19]:
#filename = 'Hugo.metaT_0.tab.out'

In [20]:
hits = defaultdict(list)





# calculate presicion and recall for all files
dfdict = {}
for inf in hmm_out:
    with open(inf) as handle:
        for queryresult in SearchIO.parse(handle, 'hmmer3-tab'):
            for hit in queryresult.hits:
                for attrib in attribs:
                    hit.accession = queryresult.id
                    hits[attrib].append(getattr(hit, attrib))
    hitdf = pd.DataFrame.from_dict(hits)



In [22]:
hitdf.head()

,accession,bias,bitscore,description,cluster_num,domain_exp_num,domain_included_num,domain_obs_num,domain_reported_num,env_num,evalue,id,overlap_num,region_num
0,TRINITY_DN374622_c0_g1_i1_1,0.1,32.3,-,0,1.0,1,1,1,1,2.100000e-10,ALL_NCBI_Palmdb_RdRp.Durnavirales.0.4.fasta,0,1
1,TRINITY_DN374622_c0_g1_i1_1,0.1,21.9,-,0,1.2,1,1,1,1,2.100000e-07,Pisuviricota.ALL_RdRp_Palmdb_0.4.fasta,0,1
2,TRINITY_DN384548_c0_g2_i1_1,0.0,54.0,-,0,1.0,1,1,1,1,5.800000e-17,ALL_NCBI_Palmdb_RdRp.Durnavirales.0.4.fasta,0,1
3,TRINITY_DN384548_c0_g2_i1_1,0.0,42.0,-,0,1.0,1,1,1,1,1.700000e-13,Pisuviricota.ALL_RdRp_Palmdb_0.4.fasta,0,1
4,TRINITY_DN384548_c0_g2_i1_1,0.0,17.0,-,0,1.0,1,1,1,1,6.000000e-06,Duplornaviricota.ALL_RdRp_Palmdb_0.4.fasta,0,1


In [24]:
idx_min_evalue = hitdf.groupby('accession')['evalue'].idxmin()

# Use the index to select the rows with the lowest 'evalue' for each 'accession'
filtered_df = hitdf.loc[idx_min_evalue]
filtered_df.to_csv('RdRp_hmmr.csv')

In [26]:
filtered_df['contig'] = filtered_df['accession'].str.rsplit('_', n=1).str[0]


In [28]:
idx_min_evalue = filtered_df.groupby('contig')['evalue'].idxmin()
df = filtered_df.loc[idx_min_evalue]


In [29]:
len(df)

5099

In [30]:
df.to_csv('RdRp_hmmr.csv')